In [ ]:
# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.5.0 tensorflow-gpu==2.5.0 opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Hand Detection using Mediapipe Holistic model

In [2]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils
actions = np.array(['an_mung', 'an_ui', 'ap_dung', 'ban_khoan', 'benh_nhan', 'be_mac', 'biet', 'bieu_tang', 'bo_y_te', 'cach_ly', 'cam_do', 'cam_on', 'can', 'chap_nhan', 'chay', 'cho', 'dau', 'day_do', 'ghet', 'giup', 'ham_mo', 'hi_sinh', 'ho', 'ket_hon', 'khai_bao', 'khau_trang', 'khu_cach_ly', 'lay_benh', 'lo_lang', 'nghi_ngoi', 'nho', 'noi_xau', 'non_oi', 'phat', 'phong_van', 'phuc_hoi', 'san_se', 'sot', 'tap_luyen', 'tham', 'that_lac', 'thich', 'thuong', 'ung_ho', 'u_benh', 'vang_loi', 'virus', 'xin_phep', 'xuat_vien', 'yeu'])

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                 
    results = model.process(image)                
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_styled_landmarks(image, results):

    # left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [5]:
def extract_keypoints(results):
    key1 = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    key2 = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

# 11. Test in Real Time

In [9]:
from tensorflow import keras
modelF= keras.models.load_model('tsl-gru-action-v2.h5')

In [10]:
colors = [(245,117,16), (117,245,16), (16,117,245),(16,17,245),(16,117,24),(17,25,160),(11,45,116),(170,205,165), (224, 32, 28), (22,142,100)]
colors = colors * 5
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [11]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = modelF.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Recognition Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
hi_sinh
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hi_sinh
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hi_sinh
<class 'mediapipe.python.solution_base.SolutionOutputs'>
biet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
biet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
biet
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cam_on
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cam_on
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cam_on
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cam_on
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cam_on
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cam_on
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cam_on
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cam_on
<class 'mediapipe.python.solution_base.SolutionOutputs'>
cam_on
<class 'mediapipe.python.solution_base.Solu

<class 'mediapipe.python.solution_base.SolutionOutputs'>
an_ui
<class 'mediapipe.python.solution_base.SolutionOutputs'>
an_ui
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<class 'mediapipe.python.solution_base.SolutionOutputs'>
chap_nhan
<cl